In [2]:
import duckdb

# Connect to a persistent database file
# If the file doesn't exist, DuckDB creates it automatically.
con = duckdb.connect('ev_data.duckdb')

print("Database connected successfully!")

Database connected successfully!


In [2]:
# Create a table directly from a CSV file
# This might take a few seconds for 24M rows, but it's a one-time cost.
con.sql("""
    CREATE OR REPLACE TABLE raw_ev_data AS 
    SELECT * FROM read_parquet('ev_data_cleaned_final.parquet');
""")

# If you have Parquet, it's even faster:
# con.sql("CREATE OR REPLACE TABLE raw_ev_data AS SELECT * FROM read_parquet('file.parquet');")

In [3]:
# Show column names and types
con.sql("DESCRIBE raw_ev_data").show()

┌───────────────────────┬──────────────┬─────────┬─────────┬─────────┬─────────┐
│      column_name      │ column_type  │  null   │   key   │ default │  extra  │
│        varchar        │   varchar    │ varchar │ varchar │ varchar │ varchar │
├───────────────────────┼──────────────┼─────────┼─────────┼─────────┼─────────┤
│ registrationYear      │ SMALLINT     │ YES     │ NULL    │ NULL    │ NULL    │
│ financialYear         │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ registrationMonthMMYY │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ makerName             │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ stateName             │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ rtoCode               │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ vehicleCategoryName   │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ vehicleModelName      │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ fuelName              │ VA

In [4]:
# View the first 5 rows
con.sql("SELECT * FROM raw_ev_data LIMIT 5").show()

┌──────────────────┬───────────────┬───────────────────────┬─────────────────────────────────┬────────────────┬─────────┬─────────────────────┬────────────────────────────────┬────────────────┬──────────────────────┬────────────────────┬─────────────────┬──────────────┬─────────────────────┐
│ registrationYear │ financialYear │ registrationMonthMMYY │            makerName            │   stateName    │ rtoCode │ vehicleCategoryName │        vehicleModelName        │    fuelName    │   vehicleClassName   │ grossVehicleWeight │  pollutionNorm  │ vehicleCount │  registrationMonth  │
│      int16       │    varchar    │        varchar        │             varchar             │    varchar     │ varchar │       varchar       │            varchar             │    varchar     │       varchar        │       int32        │     varchar     │    int16     │    timestamp_ns     │
├──────────────────┼───────────────┼───────────────────────┼─────────────────────────────────┼────────────────┼─────────┼

In [ ]:
# Count total rows
total_rows = con.sql("SELECT COUNT(*) FROM raw_ev_data").fetchall()[0][0]
print(f"Total Rows: {total_rows:,}")


# Count registrations per year to check distribution
con.sql("""
    SELECT 
        registrationYear, 
        COUNT(*) as Total_EVs
    FROM raw_ev_data
    GROUP BY 1
    ORDER BY 1 DESC
    LIMIT 10
""").show()

Total Rows: 23,973,955
┌──────────────────┬───────────┐
│ registrationYear │ Total_EVs │
│      int16       │   int64   │
├──────────────────┼───────────┤
│             2026 │    119964 │
│             2025 │   4824506 │
│             2024 │   4467461 │
│             2023 │   4501648 │
│             2022 │   4251104 │
│             2021 │   3694536 │
│             2020 │   2114736 │
└──────────────────┴───────────┘



In [17]:
con.sql("""
CREATE OR REPLACE TABLE final_data AS
SELECT
    /* =========================
       Core time dimensions
       ========================= */
    CAST(registrationMonth AS DATE)              AS registration_date,
    CAST(registrationYear AS INTEGER)            AS registration_year,
    TRIM(registrationMonthMMYY)                  AS registration_month_mmyy,
    TRIM(financialYear)                          AS financial_year,

    /* =========================
       Entity standardization
       ========================= */
    TRIM(UPPER(makerName))                       AS maker_name,
    TRIM(UPPER(stateName))                       AS state_name,
    TRIM(UPPER(rtoCode))                         AS rto_code,

    /* =========================
       Vehicle attributes
       ========================= */
    TRIM(UPPER(vehicleCategoryName))             AS vehicle_category,
    TRIM(UPPER(vehicleModelName))                AS vehicle_model,
    TRIM(UPPER(vehicleClassName))                AS vehicle_class,
    TRIM(UPPER(fuelName))                        AS fuel_type,
    TRIM(UPPER(pollutionNorm))                   AS pollution_norm,

    /* =========================
       Quantitative fields
       ========================= */
    CAST(grossVehicleWeight AS INTEGER)          AS gross_vehicle_weight,
    CAST(vehicleCount AS INTEGER)                AS quantity

    /* ❌ REMOVED: All raw column references that were duplicates */

FROM raw_ev_data
WHERE
    registrationYear IS NOT NULL
    AND registrationMonth IS NOT NULL;
""")

print("✅ Clean table created successfully!")

✅ Clean table created successfully!


In [18]:
# Check schema and first 5 rows
con.sql("DESCRIBE final_data").show()
con.sql("SELECT * FROM final_data LIMIT 5").show()

┌─────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│       column_name       │ column_type │  null   │   key   │ default │  extra  │
│         varchar         │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ registration_date       │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ registration_year       │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ registration_month_mmyy │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ financial_year          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ maker_name              │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ state_name              │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ rto_code                │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ vehicle_category        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ vehicle_model 

Analysis

In [25]:
con.sql("""
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'final_data'
ORDER BY ordinal_position;
""").show()

┌─────────────────────────┐
│       column_name       │
│         varchar         │
├─────────────────────────┤
│ registration_date       │
│ registration_year       │
│ registration_month_mmyy │
│ financial_year          │
│ maker_name              │
│ state_name              │
│ rto_code                │
│ vehicle_category        │
│ vehicle_model           │
│ vehicle_class           │
│ fuel_type               │
│ pollution_norm          │
│ gross_vehicle_weight    │
│ quantity                │
│ fuel_group              │
├─────────────────────────┤
│         15 rows         │
└─────────────────────────┘



In [ ]:

con.sql("""
ALTER TABLE final_data
ADD COLUMN fuel_group VARCHAR;
""")
con.sql("""   
UPDATE final_data
SET fuel_group =
    CASE
        -- EV
        WHEN fuel_type IN ('PURE EV', 'ELECTRIC(BOV)')
            THEN 'EV'

        -- Hybrid
        WHEN fuel_type IN (
            'STRONG HYBRID EV',
            'PLUG-IN HYBRID EV',
            'PETROL/HYBRID',
            'PETROL(E20)/HYBRID/CNG'
        )
            THEN 'HYBRID'

        -- Petrol
        WHEN fuel_type IN (
            'PETROL',
            'PETROL(E20)',
            'PETROL(E20)/LPG',
            'PETROL/METHANOL'
        )
            THEN 'PETROL'

        -- Diesel
        WHEN fuel_type IN (
            'DIESEL',
            'DUAL DIESEL/CNG',
            'DUAL DIESEL/LNG'
        )
            THEN 'DIESEL'

        -- CNG
        WHEN fuel_type IN (
            'CNG ONLY',
            'PETROL/CNG',
            'HCNG'
        )
            THEN 'CNG'

        -- Others
        WHEN fuel_type IN (
            'FUEL CELL HYDROGEN',
            'SOLAR',
            'METHANOL',
            'NOT APPLICABLE'
        )
            THEN 'OTHERS'

        ELSE NULL
    END;
""")

In [31]:
# Check all unique fuel types to find the correct label for EVs
con.sql("""
SELECT
    SUM(quantity) AS ev_2w_delhi
FROM final_data
WHERE fuel_group = 'EV'
  AND vehicle_category = 'TWO WHEELER(NT)'
  AND state_name = 'DELHI';

""").show()

┌─────────────┐
│ ev_2w_delhi │
│   int128    │
├─────────────┤
│      149570 │
└─────────────┘



In [36]:
con.sql("""
    SELECT DISTINCT maker_name
    FROM final_data
    ORDER BY maker_name
""").show(max_rows=200)


┌────────────────────────────────────────────────────┐
│                     maker_name                     │
│                      varchar                       │
├────────────────────────────────────────────────────┤
│ 3EV INDUSTRIES PVT LTD                             │
│ 3GB TECHNOLOGY PVT LTD                             │
│ 3S INDUSTRIES PRIVATE LIMITED                      │
│ A B EXCAVATORS & EARTHMOVER LTD                    │
│ A B EXCAVATORS & EARTHMOVERS PVT LTD               │
│ A D AGRO WORKS                                     │
│ A K AUTTO ELECTRICAL                               │
│ A K GUPTA & CO.                                    │
│ A K S ENG AGRO INDUSTRIES                          │
│ A N ENTERPRISES                                    │
│ A P I MOTORS PVT LTD                               │
│ A R AGRO TECH                                      │
│ A-1 SUREJA INDUSTRIES                              │
│ A. A. TRAILOR                                      │
│ A.B. AGR

In [37]:
important_columns = [
    "state_name",
    "vehicle_category",
    "vehicle_class",
    "fuel_type",
    "fuel_group",
    "maker_name",
    "pollution_norm"
]
import os

folder_path = "C:\\Users\\parvs\\OneDrive\\Documents\\EV_THESIS\\reference_unique_values"

os.makedirs(folder_path, exist_ok=True)
for col in important_columns:
    df_unique = con.sql(f"""
        SELECT DISTINCT {col}
        FROM final_data
        WHERE {col} IS NOT NULL
        ORDER BY {col}
    """).df()

    file_path = os.path.join(folder_path, f"{col}_unique_values.csv")
    df_unique.to_csv(file_path, index=False)

    print(f"Saved {len(df_unique)} unique values for {col}")


Saved 35 unique values for state_name
Saved 17 unique values for vehicle_category
Saved 76 unique values for vehicle_class
Saved 28 unique values for fuel_type
Saved 6 unique values for fuel_group
Saved 2851 unique values for maker_name
Saved 27 unique values for pollution_norm
